## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

FILTER_SIZE = 21
r = 0.25

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate

lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

hanning = np.hanning(FILTER_SIZE)
hanning2d = np.outer(hanning, hanning)

lenaSize = (FILTER_SIZE, FILTER_SIZE)
FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, lenaSize[1]]))
FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
FSpaceColsM = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols)

FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

FilterF = FreqR <= r

figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
figFilter.show()

#FilterF = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)

FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3), flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

Filter = FilterFI * hanning2d

Filter = cv2.copyMakeBorder(Filter, 0, lena.shape[0] - Filter.shape[0], 0, lena.shape[1] - Filter.shape[1], cv2.BORDER_CONSTANT, value=0)

f_Filter = cv2.dft(np.float32(Filter), flags=cv2.DFT_COMPLEX_OUTPUT)
f_Filter = np.fft.fftshift(f_Filter, [0,1])

f_Filter = np.abs(f_Filter)

f_lena = cv2.dft(np.float32(lena), flags=cv2.DFT_COMPLEX_OUTPUT)
f_lena = np.fft.fftshift(f_lena, [0,1])

f_lena = f_lena * f_Filter

f_lena = np.fft.ifftshift(f_lena, [0,1])
ifft = cv2.idft(np.float32(f_lena), flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
lenaIFFT = np.round(cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])).astype(np.uint8)

fig, axs = plt.subplots(1, 3)
fig.set_size_inches(15, 10)
axs[0].imshow(lena, 'gray')
axs[0].axis("off")
axs[0].set_title("Oryginał")

axs[1].imshow(lenaIFFT, 'gray')
axs[1].axis("off")
axs[1].set_title("Po filtracji")

axs[2].imshow(cv2.absdiff(lena, lenaIFFT), 'gray')
axs[2].axis("off")
axs[2].set_title("Różnica")

plt.show()
